In [1]:
import flair.data
import pandas as pd
from flair.data import Corpus
from flair.datasets import TREC_6
from flair.embeddings import TransformerDocumentEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer

from flair.data import Corpus
from flair.datasets import CSVClassificationCorpus
import pandas as pd
from sklearn.model_selection import train_test_split

In [7]:
df = pd.read_csv('Sentence_Annotation_Assignments_Final_Dataset.tsv', sep='	', nrows=500)  # i limited it to 500 for speed of trainig on a cpu, read all when fully training

label_type = 'satisfaction'

train_dev, test = train_test_split(df, test_size=0.2)
train, dev = train_test_split(train_dev, test_size=0.2)

def load_df_to_sentences(df: pd.DataFrame):
    sentences = []
    for index, row in df.iterrows():
        sentence = flair.data.Sentence(row['text'])
        label = str(row['satisfaction'])  # must be a string, it's a classification
        sentence.add_label(label_type, label, 1.0)
        sentences.append(sentence)
    return sentences

train_sentences = load_df_to_sentences(train)
test_sentences = load_df_to_sentences(test)
dev_sentences = load_df_to_sentences(dev)



# load corpus containing training, test and dev data and if CSV has a header, you can skip it
corpus: Corpus = Corpus(train_sentences, dev_sentences, test_sentences)



# 3. create the label dictionary
label_dict = corpus.make_label_dictionary(label_type=label_type)

# 4. initialize transformer document embeddings (many models are available)
document_embeddings = TransformerDocumentEmbeddings('distilbert-base-uncased', fine_tune=True)  # transformer embeddings are hard core, awesome, you can experiment

# 5. create the text classifier
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict, label_type=label_type)

# 6. initialize trainer
trainer = ModelTrainer(classifier, corpus)

# 7. run training with fine-tuning
trainer.fine_tune('/Users/jackw/Downloads/flair_satisfaction',
                  learning_rate=5.0e-5,  # another good one to mess with
                  mini_batch_size=4,  # increase this, higher for a cpu, don't go above 8 on a gpu, sometimes get problem
                  max_epochs=10,  # mess with this
                  )

2023-03-20 13:10:34,156 Computing label dictionary. Progress:


320it [00:00, 20463.76it/s]

2023-03-20 13:10:34,172 Dictionary created for label 'satisfaction' with 26 values: 3.0 (seen 200 times), 2.5 (seen 31 times), 2.0 (seen 26 times), 2.666666667 (seen 16 times), 3.5 (seen 7 times), 1.0 (seen 5 times), 2.333333333 (seen 4 times), 1.666666667 (seen 4 times), 1.333333333 (seen 4 times), 4.0 (seen 3 times), 1.25 (seen 2 times), 2.142857143 (seen 2 times), 1.5 (seen 2 times), 4.666666667 (seen 2 times), 3.666666667 (seen 2 times), 2.25 (seen 1 times), 2.8 (seen 1 times), 2.571428571 (seen 1 times), 4.5 (seen 1 times), 2.75 (seen 1 times)


2023-03-20 13:10:37,121 ----------------------------------------------------------------------------------------------------
2023-03-20 13:10:37,121 Model: "TextClassifier(
  (embeddings): TransformerDocumentEmbeddings(
    (model): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30523, 768)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0): TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out_featu

100%|██████████| 20/20 [00:02<00:00,  6.95it/s]

...

2023-03-20 13:25:50,696 Evaluating as a multi-label problem: False
2023-03-20 13:25:50,709 DEV : loss 3.223639965057373 - f1-score (micro avg)  0.525


2023-03-20 13:25:53,476 ----------------------------------------------------------------------------------------------------
2023-03-20 13:25:53,478 Testing using last state of model ...


100%|██████████| 25/25 [00:04<00:00,  5.45it/s]

2023-03-20 13:25:58,067 Evaluating as a multi-label problem: False
2023-03-20 13:25:58,084 0.65	0.65	0.65	0.65
2023-03-20 13:25:58,084 
Results:
- F-score (micro) 0.65
- F-score (macro) 0.0979
- Accuracy 0.65

By class:
              precision    recall  f1-score   support

         3.0     0.7209    0.9394    0.8158        66
         2.0     0.2500    0.2500    0.2500         8
         2.5     0.0000    0.0000    0.0000         6
 2.333333333     0.0000    0.0000    0.0000         3
 2.666666667     0.5000    0.5000    0.5000         2
         1.5     0.0000    0.0000    0.0000         3
         1.0     0.0000    0.0000    0.0000         3
         4.0     0.0000    0.0000    0.0000         2
 3.333333333     0.0000    0.0000    0.0000         1
 3.142857143     0.0000    0.0000    0.0000         1
        1.25     0.0000    0.0000    0.0000         1
 3.666666667     0.0000    0.0000    0.0000         1
 1.666666667     0.0000    0.0000    0.0000         1
        2.25     0.0000

{'test_score': 0.65,
 'dev_score_history': [0.5625,
  0.55,
  0.525,
  0.55,
  0.5375,
  0.525,
  0.525,
  0.525,
  0.525,
  0.525],
 'train_loss_history': [2.040735564753413,
  1.3753664969932289,
  0.7281687286973465,
  0.1892297235161095,
  0.03285864864847099,
  0.004421195524628274,
  0.0009977365751183242,
  0.000771659546671799,
  0.0006056859925593017,
  0.0005516228393844357],
 'dev_loss_history': [1.8685948848724365,
  1.8217980861663818,
  2.152174472808838,
  2.600785732269287,
  2.9774205684661865,
  2.894951581954956,
  3.1520094871520996,
  3.1452202796936035,
  3.201237201690674,
  3.223639965057373]}